<a href="https://colab.research.google.com/github/HuangruiChu/ESG-DecisionTree/blob/main/ESG_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Data Wrangling

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
clf = DecisionTreeClassifier()

In [ ]:
###show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_ESG=pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/Econ-211/main/ESG_DecisionTree/MSCI_ESG_Rating.csv')
df_ESG= df_ESG[df_ESG.Month==1]
df_ESG.head()

In [ ]:
df_YearlyReturn=pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/Econ-211/main/ESG_DecisionTree/S%26P1500_2020_Yearly_Return.csv')
df_YearlyReturn=df_YearlyReturn.dropna()
df_YearlyReturn.head()

In [ ]:
df_YearlyReturn['Return']=pd.qcut(df_YearlyReturn.Yearly_Return,3,labels=np.arange(1,4,1) )
df_YearlyReturn.head()

In [ ]:
df_YearlyReturn['Rank']=df_YearlyReturn['Return'].apply(lambda x: 'High' if x==3 else 'Average' if x==2 else 'Low')

In [ ]:
df = df_ESG.merge(df_YearlyReturn, left_on='Ticker',right_on='Symbol')
df.head()

# Build Decision Tree

In [ ]:
feature_names=['ESG_Score']
features=df[feature_names]
targets=df['Rank']
targets_names = targets.unique()

In [ ]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
#####Optimizing Decision Tree Performance
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth=2,random_state=0)

# Train Decision Tree Classifer
clf = clf.fit(train_features,train_targets)

#Predict the response for test dataset
y_pred = clf.predict(test_features)

# Compute test set accuracy  
acc = accuracy_score(y_pred, test_targets)
print("Test set accuracy: {:.2f}".format(acc))

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

In [ ]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_names,class_names=targets_names)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('ESG.png')
Image(graph.create_png())